In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('nepali.csv')

In [4]:
data

,Unnamed: 0,Sentences,Sentiment
0,0,"म एक शिक्षक , शिक्षा क्षेत्रमा रमाएको मान्छे ।...",1
1,1,म सरकारी स्कूल/कलेजमा पढेर करीब १२ बर्ष भन्दा ...,1
2,2,कति राम्रो शिव मन्दिर देख्न पाइयो कुन ठाउको हो...,1
3,3,मारुनी भन्ने वितिकै सामान्य नाचनीमा आधारित कथा...,1
4,4,यो फ्लिम हेरिसकेपछी थाहा भयो कि किन दर्सकहरुले...,1
...,...,...,...
35784,33469,कोरोना भाइरसलाई विश्व स्वास्थ्य संगठनले दियो न...,0
35785,33470,डब्ल्युएचओले दियाेको कोरोनाको नयाँ नाम कोभिड,0
35786,33471,कोरोना भाइरस संक्रमणको औपचारिक नाम अब कोभिड,0
35787,33472,विश्व स्वास्थ्य संगठन डब्लुएचओ ले नोबल कोरोना ...,0


In [5]:
data['Sentiment'].unique()

array([ 1, -1,  0], dtype=int64)

In [6]:
from keras.preprocessing.text import Tokenizer
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['Sentences'].values)

In [8]:
from keras.preprocessing.sequence import pad_sequences
X = tokenizer.texts_to_sequences(data['Sentences'].values)
X = pad_sequences(X)
print(X)

[[   0    0    0 ...  439   50  964]
 [   0    0    0 ...   41  140 1180]
 [   0    0    0 ...  266  159 1448]
 ...
 [   0    0    0 ...  392   73    1]
 [   0    0    0 ...  392  195    9]
 [   0    0    0 ...  136   72   19]]


In [9]:
X[90].shape

(48,)

In [10]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D

embed_dim = 128
lstm_out = 196

model = Sequential()
'''the Embedding layer is transforming each word in your text into a 128-dimensional vector. 
This transformation is learned during the training of your model. 
The output of the Embedding layer is a 2D vector with the length of the input sentence and the embedding dimension (128 in your case) 
for each word in the sentence.'''
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))

model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 48, 128)           256000    
                                                                 
 spatial_dropout1d (Spatial  (None, 48, 128)           0         
 Dropout1D)                                                      
                                                                 
 lstm (LSTM)                 (None, 196)               254800    
                                                                 
 dense (Dense)               (None, 3)                 591       
                                                                 
Total params: 511391 (1.95 MB)
Trainable params: 511391 (1.95 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [11]:
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

Y = pd.get_dummies(data['Sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.3, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(25052, 48) (25052, 3)
(10737, 48) (10737, 3)


In [13]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 8, batch_size=batch_size, verbose = 1)

Epoch 1/8
783/783 [==============================] - 105s 134ms/step - loss: 0.6072 - accuracy: 0.7440
Epoch 2/8
783/783 [==============================] - 108s 138ms/step - loss: 0.5807 - accuracy: 0.7538
Epoch 3/8
783/783 [==============================] - 107s 136ms/step - loss: 0.5537 - accuracy: 0.7653
Epoch 4/8
783/783 [==============================] - 96s 122ms/step - loss: 0.5235 - accuracy: 0.7783
Epoch 5/8
783/783 [==============================] - 96s 123ms/step - loss: 0.5020 - accuracy: 0.7880
Epoch 6/8
783/783 [==============================] - 96s 123ms/step - loss: 0.4765 - accuracy: 0.8001
Epoch 7/8
783/783 [==============================] - 96s 123ms/step - loss: 0.4537 - accuracy: 0.8098
Epoch 8/8
783/783 [==============================] - 96s 123ms/step - loss: 0.4310 - accuracy: 0.8216


In [16]:

import torch
# Your sentence
sentence = ""

# Tokenize the sentence
sentence = tokenizer.texts_to_sequences(sentence)

# Pad the sequence
sentence = pad_sequences(sentence, maxlen=X.shape[1])

# Get the model's prediction
prediction = model.predict(sentence)

# Print the prediction
if torch.argmax(prediction) == 1:
    print("Positive sentiment")
else:
    print("Negative sentiment")

ValueError: Unexpected result of `predict_function` (Empty batch_outputs). Please use `Model.compile(..., run_eagerly=True)`, or `tf.config.run_functions_eagerly(True)` for more information of where went wrong, or file a issue/bug to `tf.keras`.